In [ ]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer


dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name, split="train")